# Sprawdzanie dokładności modelu
Plik test_data.csv jest dostępny tylko dla jury. Zawiera dane testowe - jednakowe dla wszystkich drużyn. Jest w takim samym formacie co zbiór, który otrzymaliście wraz z zadaniem, tylko już z czystymi danymi.

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
test_data = pd.read_csv("test_data.csv")

### Miejsce na twój kod:

### Definicja modelu, funkcji pomocniczych oraz stałych

In [4]:
# build model: 66 -> 150 -> 80 -> 6
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(150, input_dim=66, activation="relu"),
        tf.keras.layers.Dense(80, activation="relu"),
        tf.keras.layers.Dense(6, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="MSE", metrics=['accuracy'])
    return model

_labels_ = ['STANDING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING', 'SITTING', 'WALKING_DOWNSTAIRS']
checkpoint_path = "model/cp.ckpt"

def index_to_label(index):
    return _labels_[index]

def list_to_labels(predictions):
    return [index_to_label(i) for i in predictions]

### Wybieramy tylko te kolumny które uwzględniają średnią i odchylenie standardowe

In [5]:
mean_cols = [col for col in test_data.columns if 'mean' in col or 'Mean' in col]
std_cols = [col for col in test_data.columns if 'std' in col or 'Std' in col]

In [6]:
df_mean = test_data[mean_cols]

In [7]:
df_std = test_data[std_cols]

In [8]:
df_new = pd.concat([df_std, df_mean], axis = 1)

### Usuwamy kolumny zawierające kąt i energię i średnią częstotliwości

In [9]:
angle_cols = [col for col in df_new.columns if 'angle' in col or 'Angle' in col]
energy_cols = [col for col in df_new.columns if 'energy' in col or 'Energy' in col]
meanFreq_cols = [col for col in df_new.columns if 'meanFreq' in col]
drop_cols = angle_cols + energy_cols + meanFreq_cols
drop_cols

['angle(X,gravityMean)',
 'angle(Y,gravityMean)',
 'angle(Z,gravityMean)',
 'angle(tBodyAccJerkMean),gravityMean)',
 'angle(tBodyAccMean,gravity)',
 'angle(tBodyGyroJerkMean,gravityMean)',
 'angle(tBodyGyroMean,gravityMean)',
 'energy-mean()',
 'fBodyAcc-meanFreq()-X',
 'fBodyAcc-meanFreq()-Y',
 'fBodyAcc-meanFreq()-Z',
 'fBodyAccJerk-meanFreq()-X',
 'fBodyAccJerk-meanFreq()-Y',
 'fBodyAccJerk-meanFreq()-Z',
 'fBodyAccMag-meanFreq()',
 'fBodyBodyAccJerkMag-meanFreq()',
 'fBodyBodyGyroJerkMag-meanFreq()',
 'fBodyBodyGyroMag-meanFreq()',
 'fBodyGyro-meanFreq()-X',
 'fBodyGyro-meanFreq()-Y',
 'fBodyGyro-meanFreq()-Z']

In [10]:
df_new = df_new.drop(drop_cols, axis=1)

In [11]:
X = df_new

### Tworzymy model i importujemy wagi

In [12]:
model = create_model()
model.load_weights(checkpoint_path)

### Generowanie przewidywanej aktywności

In [13]:
predicted = list_to_labels(np.argmax(model.predict(X), axis=1))

### Sprawdzanie dokładności:
Predicted - zmienna przechowująca przewidzianą aktywność. Poniższa linijka powinna zwrócić wartość 0-1.

In [14]:
np.mean(predicted == test_data.Activity)

0.8359191773407901